# Proyecto final de imágenes
## EL5206-2 Laboratorio de inteligencia computacional y robótica
### **Integrantes:**
### Benjamín Castro
### Melanie Peña
### **Auxiliar:**
### Jorge Zambrano
### **Profesor:**
### Carlos Navarro



#### Imports de las librerías y archivos

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import skimage
import skimage.feature as feature
import tensorflow
import pandas as pd
from pathlib import Path

# Se usa el modelo pre-entrenado VGG-16
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image

# Arregla el error de la función imshow
from google.colab.patches import cv2_imshow

In [ ]:
# JPG1
if not Path("jpg1.tar.gz").is_file():
  !wget "ftp://ftp.inrialpes.fr/pub/lear/douze/data/jpg1.tar.gz"

--2022-11-14 19:47:07--  ftp://ftp.inrialpes.fr/pub/lear/douze/data/jpg1.tar.gz
           => ‘jpg1.tar.gz’
Resolving ftp.inrialpes.fr (ftp.inrialpes.fr)... 194.199.18.221
Connecting to ftp.inrialpes.fr (ftp.inrialpes.fr)|194.199.18.221|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/lear/douze/data ... done.
==> SIZE jpg1.tar.gz ... 1141827194
==> PASV ... done.    ==> RETR jpg1.tar.gz ... done.
Length: 1141827194 (1.1G) (unauthoritative)

jpg1.tar.gz         100%[===================>]   1.06G  3.46MB/s    in 2m 58s  

2022-11-14 19:50:07 (6.13 MB/s) - ‘jpg1.tar.gz’ saved [1141827194]



In [ ]:
# JPG2
if not Path("jpg2.tar.gz").is_file():
  !wget "ftp://ftp.inrialpes.fr/pub/lear/douze/data/jpg2.tar.gz"

--2022-11-14 19:50:18--  ftp://ftp.inrialpes.fr/pub/lear/douze/data/jpg2.tar.gz
           => ‘jpg2.tar.gz’
Resolving ftp.inrialpes.fr (ftp.inrialpes.fr)... 194.199.18.221
Connecting to ftp.inrialpes.fr (ftp.inrialpes.fr)|194.199.18.221|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/lear/douze/data ... done.
==> SIZE jpg2.tar.gz ... 1701364177
==> PASV ... done.    ==> RETR jpg2.tar.gz ... done.
Length: 1701364177 (1.6G) (unauthoritative)

jpg2.tar.gz         100%[===================>]   1.58G  10.8MB/s    in 4m 14s  

2022-11-14 19:54:34 (6.38 MB/s) - ‘jpg2.tar.gz’ saved [1701364177]



In [ ]:
# Extrae los archivos tar.gz
!tar -xf jpg1.tar.gz
!tar -xf jpg2.tar.gz 

### 1. *Feature Extraction*

#### CNN

In [ ]:
# Extracción con VGG-16
model = VGG16(weights='imagenet',include_top=False)

def feat_cnn(net,img):
  """ Toma el modelo con el cual predecir y la ID la imagen en string,
  devuelve el vector de características """
  assert type(img) == str,'Entregue el nombre de la consulta en string'
  path = './jpg/'+img+'.jpg' # Ruta de acceso de la base de datos
  img_loaded = image.load_img(path, target_size = (224,224))
  x = image.img_to_array(img_loaded)
  x = np.expand_dims(x,axis=0)
  x = preprocess_input(x)
  feat = net.predict(x)
  return feat

#### LBP 

In [ ]:
def feat_lbp(img):
  """ Toma la ID de imagen en string, devuelve el vector de características"""
  assert type(img) == str,'Entregue el nombre de la consulta en string'
  path = './jpg/'+img+'.jpg' # Ruta de acceso de la base de datos
  img_loaded = image.load_img(path, color_mode = "grayscale", target_size = (224,224))
  # Cantidad de píxeles de radio a considerar
  r = 1
  p = 8
  lbp_img = feature.local_binary_pattern(img_loaded, P = p, R = r)
  (hist, _) = np.histogram(lbp_img.ravel(), bins = np.arange(0, 84), range = (0,255))

  # Normalización
  hist = hist.astype("float")
  hist /= (hist.sum() + 1e-6)
  
  return hist


In [ ]:
def feat_extract(img,ext):
  """ Toma dos strings: nombre de la imagen y tipo de extracción,
  se puede escoger clásico o usando CNN"""
  assert type(img) == str,'Entregue el nombre de la consulta en string'
  assert ext == 'classic' or ext == 'cnn','Puede escoger solo cnn o classic'
  if ext == 'classic':
    return feat_lbp(img)
  if ext == 'cnn':
    return feat_cnn(model,img)

### 2. Medida de similitud

In [ ]:
def cos_dist(x1,x2):
  """ Toma la distancia entre 2 vectores """
  norm1 = np.linalg.norm(x1)
  norm2 = np.linalg.norm(x2)
  if norm1 == 0.0 or norm2 == 0.0:
    return 0.0
  else:
    num = np.sum(x1*x2)
    den = np.sqrt(norm1)*np.sqrt(norm2)
    return num/den

### 3. Ordenar resultados por relevancia